In [2]:
import os , os.path #osを読み込む
import sys
import pandas as pd #xlsxのファイルをcsvにする
import ast
import requests
from bs4 import BeautifulSoup
import json
from urllib import request
import csv
import collections
import glob
import re

In [12]:
#評価ワード読み込み
df1 = pd.read_excel("new評価ワード.xlsx", sheet_name=0)
df1.to_csv("new評価ワード.csv", index=False, header=None, encoding='utf_8_sig')
with open("new評価ワード.csv", encoding='utf_8_sig') as fp:
    
    #tuple型の評価ワード作成
    all_list = [] #all_lstはtuple型の評価ワード全部
    reader = csv.reader(fp)
    for row in reader: #rowはキヤノンらしいのかたまり
        word_list = [] #elementを追加していったもの
        for element in row: #elementはキヤノンらしいなどの単語1つ1つ
            if element != '': #空白削除
                word_list.append(element) #word_lstに追加していく
                word_tuple = tuple(word_list) #word_tupleはword_lstをタプル型に変換
        #print(word_tuple[0])
        all_list.append(word_tuple) #all_lstに追加していく
    #print(all_lst[0])


#all_lstをキーに0を値に取るdict型作成
mydict = {} #mydictはall_lstをキーに0を値に取る
for word in all_list:
    mydict[word] = 0
#print(mydict)


In [4]:
df2 = pd.read_excel("カメラ辞書.xlsx", sheet_name=0)
df2.to_csv("カメラ辞書.csv", index=False, header=None, encoding='utf_8_sig')
with open("カメラ辞書.csv", encoding='utf_8_sig') as fp:
    camera_query = df2.query('シリーズ == "EOS"')
    siborikomi = camera_query["品名"]
#siborikomi_list = []
#for item in siborikomi:
#    siborikomi_list.append(item)
#print(siborikomi_list)
    #for hinmei in camera_query:
    #print(camera_query["品名"])

In [9]:
selected_products = []
for selects in siborikomi:
    selected_products.append(selects)
#print(selected_products)

['EOS R5_body', 'EOS R6_body']


In [29]:
for series in selected_products:
    files = glob.glob("./review/*/*/{}_*_*.txt".format(series)) #*に入るのは何でもいい
    for file in files:
        f = open(file,"r")
        review = f.read() 
#        print(review)

        count_result = {}
        for selected_product in selected_products: #選択製品リストから一つ一つを取り出す
            sum_result = {} #製品ごとに結果を格納する．初期化．
            count_result[selected_product] = sum_result
            
            skip_flag = False #Falseは0と同じ
            for tuple_list in mydict: #選択製品のカウント結果のそれぞれの鍵を取り出す
                if not tuple_list in sum_result:
                    sum_result[tuple_list] = 0
                sum_result[tuple_list] += count_result[selected_product][tuple_list]
            


                #mydictの値の部分をカウントしていく

                for item in tuple_list: #itemはキヤノンらしいなどの単語1つ1つ
                    if review.count(item) > 0: #数えた結果が0より大きかったら
                        skip_flag = True #Trueは1と同じ
                        continue #次のitemに行く

                if skip_flag == True: #skip_flagがTrueなら処理実行
                    skip_flag = False #Trueを踏んだらFalseに戻す
                    mydict[tuple_list] += 1 #mydictの値のカウントを１増やす
                    continue #次のtuple_listに行く
                f.close()
                
                
print(count_result)
                    
            #count_result[selected_product]で該当レビューのカウント結果を抜き出せる
            #Pythonのdict型では，未知鍵で呼び出した場合は新規に追加される仕様

{'EOS R5_body': {('キヤノンらしい', 'キャノンらしい', 'キャノンらしく', 'キヤノンらしさ', 'キャノンっぽい', 'キヤノンぽい', 'Canonらしい', 'Canonらしく', 'canonらしさ'): 0, ('キヤノンらしくない',): 0, ('ニコンらしい', 'ニコンらしく', 'ニコンらしさ', 'ニコンっぽい', 'ニコン機らしい', 'Ｎｉｋｏｎらしい', 'NIKONらしい', 'Nikonらしい', 'Nikonらしく'): 0, ('フジらしい', 'FUJIFILMらしさ'): 0, ('ソニーらしい', 'SONYらしい'): 0, ('パナソニックらしい',): 0, ('LUMIXらしい', 'Lumixらしい'): 0, ('手になじむ', '手になじみ', '手に馴染む', '手によく馴染み', '手に馴染んで', '馴染み'): 0, ('なじめない', 'なじめなく', '馴染めず', '馴染めなかった', '慣れません'): 0, ('直感的', '直感操作', '直感的な操作', '直感的に', '直感的な'): 0, ('複雑な操作', '直感的とは言えない'): 0, ('ひと手間', '持ち替え'): 0, ('軽量', '軽量化', '軽く', '重くない'): 0, ('重い', '重く', '重たい', '重さ', '重め'): 0, ('小さい', '小さく', '小型', '小さすぎる'): 0, ('大きい', '大きく', '大きめ', 'でかい', 'でかく', 'デカく', '大きすぎる'): 0, ('使いやすい', '使い易い', '使いやすく', '使い易さ', '使い易く', '使用しやすい', 'やりやすい', '扱いやすい', '容易に操作', '優れた操作性', '快適な操作性'): 0, ('使いづらい', '使い辛い', '使い難い', '使いにくく', 'やりづらい', 'やりにくい', 'しづらい', 'し辛い', 'しにくい', '使いこなせない', '難しい'): 0, ('握りやすい', '握りやすく', '握り易く', '握りやすさ', '握りやすかった', '持ちやすい', 'つかみやすい', 'しっかり握れ', 'しっかり持てる')

In [84]:
sum_result = {} #製品ごとに結果を格納する．初期化．
for selected_prooduct in selected_prooducts: #選択製品リストから一つ一つを取り出す
    for all_list in mydict: #選択製品のカウント結果のそれぞれの鍵を取り出す
        if not all_list in sum_result:
            sum_result[all_list] = 0
            sum_result[all_list] += count_result[selected_product][all_list]
        #count_result[selected_product]で該当レビューのカウント結果を抜き出せる
        #Pythonのdict型では，未知鍵で呼び出した場合は新規に追加される仕様

NameError: name 'count_result' is not defined

In [73]:
#txtファイルを読みこむ
files = glob.glob("./review/*/*/{}_*_*.txt".format(series)) #*に入るのは何でもいい
for file in files:
    f = open(file,"r")
    review = f.read() 

    #mydictの値の部分をカウントしていく
    skip_flag = False #Falseは0と同じ
    for tuple_list in mydict: #tuple_listはキヤノンらしいのかたまり
        for item in tuple_list: #itemはキヤノンらしいなどの単語1つ1つ
            if review.count(item) > 0: #数えた結果が0より大きかったら
                skip_flag = True #Trueは1と同じ
                continue #次のitemに行く

        if skip_flag == True: #skip_flagがTrueなら処理実行
            skip_flag = False #Trueを踏んだらFalseに戻す
            mydict[tuple_list] += 1 #mydictの値のカウントを１増やす
            continue #次のtuple_listに行く
        f.close()

['./review\\mirrorless\\Canon\\EOS R5_body_1_1.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_10.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_11.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_12.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_13.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_14.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_15.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_2.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_3.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_4.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_5.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_6.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_7.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_8.txt', './review\\mirrorless\\Canon\\EOS R5_body_1_9.txt', './review\\mirrorless\\Canon\\EOS R5_body_2_1.txt', './review\\mirrorless\\Canon\\EOS R5_body_2_10.txt', './review\\mirrorless\\Canon\\EOS R5_body_2_11.txt', './review\\mirrorless\\Canon\\EOS R5_body_2_12.txt', '.

In [70]:
file = open("評価ワード出現率＿シリーズ.csv", "w", encoding = 'cp932', errors = 'ignore', newline="")
writer = csv.writer(file)
mylist = list(mydict.values())
keisan_list = []
name_list = []
    
for mykey,myvalue in mydict.items():
    keisan = myvalue*100/len(files)
    keisan_list.append(keisan)
    name_list.append(mykey[0])
    
writer.writerow(name_list)
writer.writerow(mylist)
writer.writerow(keisan_list)
file.close()
#print(mylist)